In [31]:
import numpy as np
import pandas as pd
from pathlib import Path
from math import radians, cos, sin, asin, sqrt
import datetime as dt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold,  KFold,train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", 100)

from google.colab import drive
drive.mount('/content/gdrive')
data_path = "/content/gdrive/My Drive/FareClassification/"

train_df = pd.read_csv(data_path + "train.csv",index_col="tripid" )
test_df = pd.read_csv( data_path + "test.csv",index_col="tripid")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


1.Preprocessing & Feature Engineering




In [32]:
train_df['calculated_duration'] = (pd.to_datetime(train_df['drop_time']) - pd.to_datetime(train_df['pickup_time']))/np.timedelta64(1,'s')
train_df['duration'] = train_df['duration'].fillna(train_df ['calculated_duration'])
train_df = train_df.drop(train_df[['calculated_duration','pickup_time','drop_time']], axis=1)
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,1020.0,NaN,NaN,NaN,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [33]:
correct = []
incorrect =[]
for value in train_df.values:
  if value[-1] == 'correct':
    correct.append(value)
  else:
    incorrect.append(value)
    
correct = pd.DataFrame(correct,columns=train_df.columns)
incorrect = pd.DataFrame(incorrect,columns=train_df.columns)

correct = correct.fillna(correct.mean())
incorrect = incorrect.fillna(incorrect.mean())
train_df = pd.concat([correct,incorrect])

In [34]:
dataset_map = {"incorrect": 0, "correct":1}
train_df['label'] = train_df["label"].map(dataset_map)
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,10.50000,834.0,56.000000,0.000000,64.000000,6.86252,79.8993,6.90330,79.8783,270.32,1
1,10.50000,791.0,47.000000,0.000000,134.000000,6.88589,79.8984,6.91373,79.8923,197.85,1
2,10.50000,1087.0,80.000000,0.000000,61.000000,6.90839,79.8651,6.93669,79.9146,301.64,1
3,10.50000,598.0,271.000000,15.663800,68.000000,6.92570,79.8895,6.92748,79.8971,82.30,1
4,12.44434,1020.0,356.655679,16.909938,107.549152,6.87441,79.8615,6.84478,79.9290,358.39,1


In [35]:
def calc_haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2]) 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371
    return c * r

train_df['distance'] = calc_haversine(train_df['pick_lat'], train_df['pick_lon'], train_df['drop_lat'] , train_df['drop_lon'])
train_df = train_df.drop(train_df[['pick_lat','pick_lon','drop_lat','drop_lon']], axis=1)
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,distance
0,10.50000,834.0,56.000000,0.000000,64.000000,270.32,1,2.467063
1,10.50000,791.0,47.000000,0.000000,134.000000,197.85,1,0.868942
2,10.50000,1087.0,80.000000,0.000000,61.000000,301.64,1,5.531798
3,10.50000,598.0,271.000000,15.663800,68.000000,82.30,1,0.845795
4,12.44434,1020.0,356.655679,16.909938,107.549152,358.39,1,7.527884


In [36]:
train_df = train_df.drop(train_df[['meter_waiting_till_pickup']], axis=1)

In [37]:
train_df['distance_per_unitfare'] = train_df['distance']/train_df['fare']
train_df.loc[~np.isfinite(train_df['distance_per_unitfare'] ), 'distance_per_unitfare'] = 0
train_df ['feature2'] = (train_df['duration']- train_df['meter_waiting'])/train_df['fare']
train_df.loc[~np.isfinite(train_df['feature2'] ), 'feature2'] = 0
train_df.head(3)

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,label,distance,distance_per_unitfare,feature2
0,10.5,834.0,56.0,0.0,270.32,1,2.467063,0.009126,2.878070
1,10.5,791.0,47.0,0.0,197.85,1,0.868942,0.004392,3.760425
2,10.5,1087.0,80.0,0.0,301.64,1,5.531798,0.018339,3.338417


In [38]:
from collections import Counter
# Outlier detection 
def detect_outliers(df,n,features):
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | 
                              (df[col] > Q3 + outlier_step )].index
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
   
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)  

    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    return multiple_outliers   

# detect outliers from distance and fare
Outliers_to_drop = detect_outliers(train_df,2,["distance","fare"])
train_df.loc[Outliers_to_drop]

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,label,distance,distance_per_unitfare,feature2
20,10.5,1443.0,59.0,0.000000,421.11,1,8.829172,0.020966,3.286552
20,10.5,38.0,20.0,0.000000,764.69,0,14.026966,0.018343,0.023539
209,10.5,1971.0,380.0,0.000000,371.48,1,8.573144,0.023078,4.282869
209,10.5,4565.0,673.0,39.235900,2393.70,0,16.768252,0.007005,1.625935
247,10.5,5119.0,2565.0,148.257000,759.50,1,10.497586,0.013822,3.362739
247,10.5,2978.0,670.0,0.000000,654.04,0,14.202696,0.021715,3.528836
436,10.5,1736.0,49.0,0.000000,510.66,1,10.052934,0.019686,3.303568
436,10.5,1683.0,516.0,29.824800,1008.23,0,19.425756,0.019267,1.157474
616,10.5,3700.0,482.0,27.859600,768.78,1,12.509449,0.016272,4.185853
616,10.5,65815.0,50125.0,2922.287500,6630.02,0,16.160057,0.002437,2.366509


In [39]:
train_df = train_df.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

In [40]:
train_df.isnull().sum()

additional_fare          0
duration                 0
meter_waiting            0
meter_waiting_fare       0
fare                     0
label                    0
distance                 0
distance_per_unitfare    0
feature2                 0
dtype: int64

In [41]:
X_train = train_df.drop(['label'],axis='columns')
Y_train = train_df['label']
Y_train.head()

0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64

2.Model creation & Training

In [ ]:
random_grid = {'n_estimators': range(100,500,100),
               'max_depth': range(5,20,1),
               'min_samples_leaf':range(2,5,1),
              'max_features':['auto','sqrt','log2'],
              'bootstrap': [True, False],
              'min_samples_split': range(2,5,1)}

rforest = RandomForestClassifier()
xgb_= XGBClassifier()

rfcv = RandomizedSearchCV(rforest, random_grid,cv=5)
xgbcv = RandomizedSearchCV(xgb_, random_grid,cv=5)

rfcv.fit(X_train, Y_train)
xgbcv.fit(X_train, Y_train)

# Print the tuned parameters and score
print("Tuned Parameters for random forest: {}".format(rfcv.best_params_))
print("Best score for random forest is {}".format(rfcv.best_score_))
print("Tuned Parameters for xgboost: {}".format(xgbcv.best_params_))
print("Best score for xgboost is {}".format(xgbcv.best_score_))


Tuned Parameters for random forest: {'n_estimators': 100, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 18, 'bootstrap': False}
Best score for random forest is 0.9441659253083626
Tuned Parameters for xgboost: {'n_estimators': 200, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 13, 'bootstrap': False}
Best score for xgboost is 0.9450975119168052


In [42]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

numeric_features = train_df.select_dtypes(include=['int64', 'float64']).drop(['label'], axis=1).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

level_zero = list()
level_zero.append(('xg',XGBClassifier(n_estimators=550,subsample=0.14))) 
level_zero.append(('mlp',MLPClassifier(hidden_layer_sizes=(50,100,50), max_iter=1000)))
level_zero.append(('dt', RandomForestClassifier(n_estimators = 100, max_features = 'log2')))
level_one = LogisticRegression(penalty="l2", C=3)
 
estimator = StackingClassifier(estimators=level_zero, final_estimator=level_one, cv=10)

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', estimator)])

In [43]:
x_train, x_eval, y_eval, y_eval = train_test_split(X_train, Y_train, test_size=0.25,stratify=Y_train)

In [44]:
f1_score(y_eval, y_pred,average='weighted')

0.8298654575769772

In [45]:
pipe.fit(X_train,Y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                               

3.Test Data Preprocessing

In [46]:
test_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [47]:
test_df['distance'] = calc_haversine(test_df['pick_lat'], test_df['pick_lon'], test_df['drop_lat'] , test_df['drop_lon'])

In [48]:
test_df = test_df.drop(['pick_lat','pick_lon','drop_lat','drop_lon'],axis='columns')

In [49]:
test_df = test_df.drop(['pickup_time','drop_time','meter_waiting_till_pickup'],axis='columns')

In [50]:
test_df['distance_per_unitfare'] = test_df['distance']/test_df['fare']
test_df.loc[~np.isfinite(test_df['distance_per_unitfare'] ), 'distance_per_unitfare'] = 0
test_df ['feature2'] = (test_df['duration']- test_df['meter_waiting'])/test_df['fare']
test_df.loc[~np.isfinite(test_df['feature2'] ), 'feature2'] = 0
test_df.head(3)

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,distance,distance_per_unitfare,feature2
tripid,,,,,,,,
213284604,10.5,924,42,2.4486,289.27,1.536052,0.005310,3.049055
213286352,10.5,4249,20,0.0000,1912.70,12.990640,0.006792,2.211011
213293973,10.5,1552,255,2.6588,394.00,5.690729,0.014443,3.291878


In [51]:
test_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,distance,distance_per_unitfare,feature2
tripid,,,,,,,,
213284604,10.5,924,42,2.4486,289.27,1.536052,0.005310,3.049055
213286352,10.5,4249,20,0.0000,1912.70,12.990640,0.006792,2.211011
213293973,10.5,1552,255,2.6588,394.00,5.690729,0.014443,3.291878
213294622,10.5,462,16,0.0000,154.32,0.585046,0.003791,2.890098
213298687,10.5,814,392,12.3692,147.47,2.408311,0.016331,2.861599


4.Predicting Results



In [52]:
result = classifier.predict(test_df)
test_df['prediction'] = result

In [54]:
final_df = test_df.drop(['additional_fare','distance','duration','fare','meter_waiting','meter_waiting_fare','distance_per_unitfare','feature2'],axis='columns')

In [55]:
final_df.head()

,prediction
tripid,
213284604,1
213286352,0
213293973,1
213294622,1
213298687,1


5.Write Generated csv file  to the drive

In [56]:
final_df.to_csv(data_path+'submission.csv', index=True)